In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from datetime import date


# DATA Preparation

In [2]:
applicant_df = pd.read_csv(
    Path('/Users/ameerirfan/Desktop/FT-BTC-Activities/FT-BTC_Unit_13/applicants_data.csv',)
)

applicant_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
applicant_df.dtypes

EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [4]:
applicant_df = applicant_df.drop(columns=['EIN','NAME'])

applicant_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
cat_variables = list(applicant_df.dtypes[applicant_df.dtypes=='object'].index)

cat_variables

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [6]:
enc = OneHotEncoder(sparse=False)

enc_data = enc.fit_transform(applicant_df[cat_variables])

In [7]:
enc_df = pd.DataFrame(
    enc_data,
    columns=enc.get_feature_names(cat_variables)
)

enc_df.head()

/Users/ameerirfan/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [8]:
non_cat_variables = list(applicant_df.dtypes[applicant_df.dtypes!='object'].index)
non_cat_variables

['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL']

In [9]:
enc_df = pd.concat([applicant_df[non_cat_variables], enc_df], axis=1)

enc_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
Y = enc_df['IS_SUCCESSFUL']
Y.head()

0    1
1    1
2    0
3    1
4    1
Name: IS_SUCCESSFUL, dtype: int64

In [11]:
X = enc_df.drop(columns=['IS_SUCCESSFUL'])
X.head()

,STATUS,ASK_AMT,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)

In [13]:
scaler = StandardScaler()

scaler_X = scaler.fit(X_train)

X_train_scaler = scaler_X.transform(X_train)

X_test_scaler = scaler_X.transform(X_test)

# BCM Using a Neural Network

In [14]:
num_input_feat = X.shape[1]
num_input_feat

116

In [15]:
num_output_neurons = 1

In [16]:
hidden_nodes_layer1 = np.ceil(np.sqrt(num_input_feat * num_output_neurons))
hidden_nodes_layer1

11.0

In [17]:
hidden_nodes_layer2 =  np.ceil(np.sqrt(hidden_nodes_layer1 * num_output_neurons))
hidden_nodes_layer2

4.0

In [18]:
nn = Sequential()

2022-08-27 19:29:01.029752: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
nn.add(
    Dense(
        units=hidden_nodes_layer1,
        activation='relu',
        input_dim=num_input_feat
    )
)

In [20]:
nn.add(
    Dense(
        units=hidden_nodes_layer2,
        activation='relu'
    )
)

In [21]:
nn.add(
    Dense(
        units=1,
        activation='sigmoid'
    )
)

In [22]:
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 11)                1287      
                                                                 
 dense_1 (Dense)             (None, 4)                 48        
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
Total params: 1,340
Trainable params: 1,340
Non-trainable params: 0
_________________________________________________________________


In [23]:
nn.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [24]:
model_nn=nn.fit(X_train_scaler, Y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 1s 872us/step - loss: 0.6142 - accuracy: 0.6971
Epoch 2/50
804/804 [==============================] - 1s 862us/step - loss: 0.5639 - accuracy: 0.7291
Epoch 3/50
804/804 [==============================] - 1s 844us/step - loss: 0.5554 - accuracy: 0.7308
Epoch 4/50
804/804 [==============================] - 1s 824us/step - loss: 0.5515 - accuracy: 0.7313
Epoch 5/50
804/804 [==============================] - 1s 806us/step - loss: 0.5499 - accuracy: 0.7321
Epoch 6/50
804/804 [==============================] - 1s 812us/step - loss: 0.5486 - accuracy: 0.7312
Epoch 7/50
804/804 [==============================] - 1s 807us/step - loss: 0.5473 - accuracy: 0.7332
Epoch 8/50
804/804 [==============================] - 1s 810us/step - loss: 0.5462 - accuracy: 0.7334
Epoch 9/50
804/804 [==============================] - 1s 809us/step - loss: 0.5461 - accuracy: 0.7334
Epoch 10/50
804/804 [==============================] - 1s 823us/step - loss: 0.544

In [25]:
model_loss, model_accuracy = nn.evaluate(
                                        X_test_scaler,
                                        Y_test,
                                        verbose=2
)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5611 - accuracy: 0.7286 - 238ms/epoch - 890us/step
Loss: 0.5610794425010681, Accuracy: 0.7286297082901001


In [26]:
Path = Path('/Users/ameerirfan/Desktop/FT-BTC-Activities/FT-BTC_Unit_13/AlphabetSoup.h5')
nn.save(Path)

In [27]:
results_df=pd.DataFrame(
                    data={
                        'Date': [date.today()],
                        'Model'   : [0],
                        'Observations': ['Original model, geometric mean for number of layers'],
                        'Loss'    : [model_loss], 
                        'Accuracy': [model_accuracy],
                        'Hidden layers': 2,
                        'Input Features':[num_input_feat],
                        'Output Neurons': [num_output_neurons],
                        'Neurons in Hidden Layers1' : [hidden_nodes_layer1],
                        'Neurons in Hidden Layers2' : [hidden_nodes_layer2],
                        'Epochs'         : [50],
                        'Activation Hidden layer 1': ['relu'],
                        'Activation Hidden layer 2': ['relu'],
                        'Activation Output layer 1': ['sigmoid'],
                        'loss':['binary_crossentropy'],
                        'optimizer':['adam'],
                        'metrics':['accuracy']          
                    })

print(results_df.T)

                                                                           0
Date                                                              2022-08-27
Model                                                                      0
Observations               Original model, geometric mean for number of l...
Loss                                                                0.561079
Accuracy                                                             0.72863
Hidden layers                                                              2
Input Features                                                           116
Output Neurons                                                             1
Neurons in Hidden Layers1                                               11.0
Neurons in Hidden Layers2                                                4.0
Epochs                                                                    50
Activation Hidden layer 1                                               relu

In [28]:
results_df.to_csv('/Users/ameerirfan/Desktop/FT-BTC-Activities/FT-BTC_Unit_13/results.csv', mode='a', header=True) 
results_df=[]